In [1]:
import pandas as pd
import numpy as np

In [2]:
ubic = pd.read_excel('DatosQF78.xlsx',sheet_name = 'Ubicación Cajeros')
retiro = pd.read_excel('DatosQF78.xlsx',sheet_name = 'Retiros')

In [3]:
ubic

,Cajero,Coordenada en X,Coordenada en Y,Costo Fijo por Stock Out,Costo Variable por Stock Out,Duración de la recarga [Horas],Lunes,Martes,Miércoles,Jueves,Viernes,Sábado,Domingo,Turno mañana,Turno tarde,Turno noche
0,Cajero 1,50,104,0.39,0.04,0.2,1,1,1,1,1,1,1,0,0,1
1,Cajero 2,24,24,0.37,0.02,0.2,0,1,1,1,1,1,0,1,1,1
2,Cajero 3,9,48,0.44,0.05,0.3,1,1,1,1,0,0,1,1,1,1
3,Cajero 4,79,73,0.30,0.05,0.2,1,1,0,1,1,0,1,1,1,1
4,Cajero 5,13,20,0.23,0.04,0.3,1,1,0,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,Cajero 227,78,18,0.40,0.05,0.3,0,1,1,1,0,1,1,1,1,1
227,Cajero 228,35,23,0.26,0.04,0.3,1,0,1,0,1,1,1,1,1,1
228,Cajero 229,117,79,0.34,0.02,0.2,1,0,1,1,1,1,1,0,1,1
229,Cajero 230,35,44,0.18,0.05,0.2,0,1,1,1,1,0,1,0,1,0


In [4]:
ubic.isna().sum()

Cajero                            0
Coordenada en X                   0
Coordenada en Y                   0
Costo Fijo por Stock Out          0
Costo Variable por Stock Out      0
Duración de la recarga [Horas]    0
Lunes                             0
Martes                            0
Miércoles                         0
Jueves                            0
Viernes                           0
Sábado                            0
Domingo                           0
Turno mañana                      0
Turno tarde                       0
Turno noche                       0
dtype: int64

In [5]:
retiro

,Día,Cajero,Monto,Hora de la transacción
0,1,Cajero 196,2.70,0:0
1,1,Cajero 14,2.92,0:1
2,1,Cajero 143,5.69,0:3
3,1,Cajero 142,6.74,0:4
4,1,Cajero 214,2.40,0:6
...,...,...,...,...
98217,102,Cajero 37,5.04,23:39
98218,102,Cajero 49,2.60,23:42
98219,102,Cajero 209,3.03,23:42
98220,102,Cajero 173,6.07,23:44


In [6]:
retiro.isna().sum()

Día                       0
Cajero                    0
Monto                     0
Hora de la transacción    0
dtype: int64

## Pregunta 1
Considerando los primeros 80 días de retiros en los cajeros (inclusive el día 80), ¿cuáles son los cajeros con más y con menos cantidad de visitas (consideraremos que cada registro es una visita)?

In [7]:
filtro1 = retiro[retiro['Día']<=80]
cajero_visita = filtro1.groupby('Cajero')['Hora de la transacción'].count()
cajero_visita = cajero_visita.reset_index().sort_values('Hora de la transacción', ascending = False)
cajero_visita.rename(columns={'Hora de la transacción': 'Cantidad de visitas'}, inplace=True)
cajero_visita

,Cajero,Cantidad de visitas
91,Cajero 181,379
217,Cajero 87,375
84,Cajero 175,375
97,Cajero 187,375
94,Cajero 184,374
...,...,...
144,Cajero 229,296
105,Cajero 194,295
192,Cajero 64,294
142,Cajero 227,276


## Pregunta 2
Si se asume que todos los cajeros se encuentran, al comienzo del día 1 de nuestro registro, con 120 unidades monetarias de stock de dinero, ¿cuáles son los cajeros que primero quedarán sin stock, si se consideran como retiros de dinero, los retiros que están registrados históricamente?

In [8]:
retiros2 = retiro.groupby(['Día','Cajero'])['Monto'].sum().reset_index()
retiros2[retiros2['Cajero']=='Cajero 1']

,Día,Cajero,Monto
0,1,Cajero 1,10.36
228,2,Cajero 1,22.22
456,3,Cajero 1,3.65
684,4,Cajero 1,13.18
912,5,Cajero 1,12.98
...,...,...,...
22044,98,Cajero 1,8.30
22270,99,Cajero 1,7.62
22500,100,Cajero 1,7.54
22726,101,Cajero 1,12.56


In [9]:
def stockout(data,dia,cajero):
    stock_act = data[(data['Día']<=dia)&(data['Cajero']==cajero)].groupby(['Cajero'])['Monto'].sum()[0]
    return stock_act

In [10]:
retiros2

,Día,Cajero,Monto
0,1,Cajero 1,10.36
1,1,Cajero 10,6.68
2,1,Cajero 100,15.89
3,1,Cajero 101,19.57
4,1,Cajero 102,7.47
...,...,...,...
23178,102,Cajero 95,29.13
23179,102,Cajero 96,30.82
23180,102,Cajero 97,26.44
23181,102,Cajero 98,24.96


In [11]:
retiros2['Stockout'] = retiros2.apply(lambda x:stockout(retiros2,x['Día'],x['Cajero']),axis=1)

In [12]:
filtro2 = retiros2[retiros2['Cajero']=='Cajero 1']

In [13]:
filtro2

,Día,Cajero,Monto,Stockout
0,1,Cajero 1,10.36,10.36
228,2,Cajero 1,22.22,32.58
456,3,Cajero 1,3.65,36.23
684,4,Cajero 1,13.18,49.41
912,5,Cajero 1,12.98,62.39
...,...,...,...,...
22044,98,Cajero 1,8.30,957.65
22270,99,Cajero 1,7.62,965.27
22500,100,Cajero 1,7.54,972.81
22726,101,Cajero 1,12.56,985.37


In [14]:
retiros2[retiros2['Stockout']>=120].head(20)

,Día,Cajero,Monto,Stockout
693,4,Cajero 107,76.40,126.19
695,4,Cajero 109,60.84,135.60
697,4,Cajero 110,36.52,121.29
708,4,Cajero 120,21.70,124.93
709,4,Cajero 121,54.08,121.02
763,4,Cajero 171,20.48,122.98
766,4,Cajero 174,32.93,123.37
770,4,Cajero 178,47.09,130.37
805,4,Cajero 209,31.11,141.49
812,4,Cajero 216,24.94,121.97


# Pregunta 3

Considerando solamente los cajeros que NO pueden ser reabastecidos ni sábado ni domingo, ¿qué podemos decir sobre estos cajeros?

In [15]:
ubic[(ubic['Sábado']==0)&(ubic['Domingo']==0)]

,Cajero,Coordenada en X,Coordenada en Y,Costo Fijo por Stock Out,Costo Variable por Stock Out,Duración de la recarga [Horas],Lunes,Martes,Miércoles,Jueves,Viernes,Sábado,Domingo,Turno mañana,Turno tarde,Turno noche
16,Cajero 17,25,33,0.20,0.02,0.4,1,1,1,1,1,0,0,0,1,1
29,Cajero 30,51,93,0.22,0.02,0.3,1,1,1,0,0,0,0,0,0,1
51,Cajero 52,38,86,0.23,0.03,0.4,1,1,0,1,0,0,0,1,1,1
60,Cajero 61,80,112,0.31,0.03,0.4,1,1,1,1,0,0,0,0,1,1
72,Cajero 73,108,7,0.24,0.05,0.3,0,0,1,0,1,0,0,1,0,1
147,Cajero 148,61,17,0.29,0.05,0.2,1,1,0,1,1,0,0,1,0,1
149,Cajero 150,105,87,0.34,0.03,0.4,1,1,1,1,1,0,0,0,1,1
166,Cajero 167,78,103,0.23,0.03,0.4,0,1,1,1,0,0,0,1,1,1
169,Cajero 170,117,9,0.35,0.03,0.2,0,1,1,1,1,0,0,1,1,1
174,Cajero 175,58,77,0.28,0.02,0.3,1,1,1,1,0,0,0,1,0,1


In [16]:
estos_cajeros= ubic[(ubic['Sábado']==0)&(ubic['Domingo']==0)]
estos_cajeros['Duración de la recarga [Horas]'].count()

14

In [17]:
estos_cajeros.describe()

,Coordenada en X,Coordenada en Y,Costo Fijo por Stock Out,Costo Variable por Stock Out,Duración de la recarga [Horas],Lunes,Martes,Miércoles,Jueves,Viernes,Sábado,Domingo,Turno mañana,Turno tarde,Turno noche
count,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.0,14.0,14.000000,14.000000,14.000000
mean,66.928571,58.857143,0.247143,0.031429,0.314286,0.714286,0.714286,0.857143,0.857143,0.571429,0.0,0.0,0.642857,0.642857,0.928571
std,34.898676,40.839003,0.059798,0.009493,0.077033,0.468807,0.468807,0.363137,0.363137,0.513553,0.0,0.0,0.497245,0.497245,0.267261
min,4.000000,4.000000,0.160000,0.020000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
25%,41.250000,18.250000,0.205000,0.030000,0.300000,0.250000,0.250000,1.000000,1.000000,0.000000,0.0,0.0,0.000000,0.000000,1.000000
50%,68.500000,75.500000,0.235000,0.030000,0.300000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,0.0,1.000000,1.000000,1.000000
75%,98.750000,91.500000,0.287500,0.030000,0.400000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,0.0,1.000000,1.000000,1.000000
max,117.000000,112.000000,0.350000,0.050000,0.400000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,0.0,1.000000,1.000000,1.000000


In [18]:
estos_cajeros_semana = estos_cajeros[((estos_cajeros['Lunes']+estos_cajeros['Martes']+estos_cajeros['Miércoles']+estos_cajeros['Jueves']+estos_cajeros['Viernes'])>=4)]
estos_cajeros_semana

,Cajero,Coordenada en X,Coordenada en Y,Costo Fijo por Stock Out,Costo Variable por Stock Out,Duración de la recarga [Horas],Lunes,Martes,Miércoles,Jueves,Viernes,Sábado,Domingo,Turno mañana,Turno tarde,Turno noche
16,Cajero 17,25,33,0.20,0.02,0.4,1,1,1,1,1,0,0,0,1,1
60,Cajero 61,80,112,0.31,0.03,0.4,1,1,1,1,0,0,0,0,1,1
147,Cajero 148,61,17,0.29,0.05,0.2,1,1,0,1,1,0,0,1,0,1
149,Cajero 150,105,87,0.34,0.03,0.4,1,1,1,1,1,0,0,0,1,1
169,Cajero 170,117,9,0.35,0.03,0.2,0,1,1,1,1,0,0,1,1,1
174,Cajero 175,58,77,0.28,0.02,0.3,1,1,1,1,0,0,0,1,0,1
206,Cajero 207,76,4,0.16,0.03,0.3,1,0,1,1,1,0,0,1,1,1
215,Cajero 216,4,100,0.25,0.04,0.3,1,1,1,1,1,0,0,1,0,1


In [19]:
estos_cajeros_semana['Duración de la recarga [Horas]'].count()

8

In [20]:
estos_cajeros_semana1 = estos_cajeros[((estos_cajeros['Lunes']+estos_cajeros['Martes']+estos_cajeros['Miércoles']+estos_cajeros['Jueves']+estos_cajeros['Viernes'])<=2)]
estos_cajeros_semana1

,Cajero,Coordenada en X,Coordenada en Y,Costo Fijo por Stock Out,Costo Variable por Stock Out,Duración de la recarga [Horas],Lunes,Martes,Miércoles,Jueves,Viernes,Sábado,Domingo,Turno mañana,Turno tarde,Turno noche
72,Cajero 73,108,7,0.24,0.05,0.3,0,0,1,0,1,0,0,1,0,1


## Pregunta 4

¿Y si ahora queremos mirar la cantidad de cajeros que es abastecido cada día?

In [21]:
data_lunes = ubic.groupby('Lunes')['Cajero'].count().reset_index()
data_lunes

,Lunes,Cajero
0,0,72
1,1,159


In [22]:
dias = ['Lunes','Martes','Miércoles','Jueves','Viernes','Sábado','Domingo']
reab = {}
for dia in dias:
    data = ubic.groupby(dia)['Cajero'].count().reset_index()
    cant = data.loc[1,'Cajero']
    reab[dia] = cant
    
print(f'El día en que más cajeros pueden ser reabastecidos es el día {max(reab)} con {reab[max(reab)]} cajeros' )

El día en que más cajeros pueden ser reabastecidos es el día Viernes con 177 cajeros


## Pregunta 5

Se define un cajero como “matutino” si la cantidad de transacciones históricas (considerando la totalidad de retiros que están registrados), para ese cajero, es mayor entre las 0hrs y las 11:59hrs que entre las 12hrs y las 23:59hrs. 

Bajo esta definición, ¿Cuántos y cuáles son los cajeros “matutinos”?

In [23]:
retiro

,Día,Cajero,Monto,Hora de la transacción
0,1,Cajero 196,2.70,0:0
1,1,Cajero 14,2.92,0:1
2,1,Cajero 143,5.69,0:3
3,1,Cajero 142,6.74,0:4
4,1,Cajero 214,2.40,0:6
...,...,...,...,...
98217,102,Cajero 37,5.04,23:39
98218,102,Cajero 49,2.60,23:42
98219,102,Cajero 209,3.03,23:42
98220,102,Cajero 173,6.07,23:44


In [24]:
retiro5 = retiro
retiro5['Hora de la transacción'] = pd.to_datetime(retiro5['Hora de la transacción'],format="%H:%M")
retiro5

,Día,Cajero,Monto,Hora de la transacción
0,1,Cajero 196,2.70,1900-01-01 00:00:00
1,1,Cajero 14,2.92,1900-01-01 00:01:00
2,1,Cajero 143,5.69,1900-01-01 00:03:00
3,1,Cajero 142,6.74,1900-01-01 00:04:00
4,1,Cajero 214,2.40,1900-01-01 00:06:00
...,...,...,...,...
98217,102,Cajero 37,5.04,1900-01-01 23:39:00
98218,102,Cajero 49,2.60,1900-01-01 23:42:00
98219,102,Cajero 209,3.03,1900-01-01 23:42:00
98220,102,Cajero 173,6.07,1900-01-01 23:44:00


In [25]:
retiro5['Segundos']= retiro5['Hora de la transacción'].dt.minute*60 + retiro5['Hora de la transacción'].dt.hour*3600
retiro5

,Día,Cajero,Monto,Hora de la transacción,Segundos
0,1,Cajero 196,2.70,1900-01-01 00:00:00,0
1,1,Cajero 14,2.92,1900-01-01 00:01:00,60
2,1,Cajero 143,5.69,1900-01-01 00:03:00,180
3,1,Cajero 142,6.74,1900-01-01 00:04:00,240
4,1,Cajero 214,2.40,1900-01-01 00:06:00,360
...,...,...,...,...,...
98217,102,Cajero 37,5.04,1900-01-01 23:39:00,85140
98218,102,Cajero 49,2.60,1900-01-01 23:42:00,85320
98219,102,Cajero 209,3.03,1900-01-01 23:42:00,85320
98220,102,Cajero 173,6.07,1900-01-01 23:44:00,85440


In [26]:
mañana = retiro5[(retiro5['Segundos']>=0)&(retiro5['Segundos']<12*3600)].groupby('Cajero')['Día'].count().reset_index()
mañana

,Cajero,Día
0,Cajero 1,111
1,Cajero 10,214
2,Cajero 100,217
3,Cajero 101,209
4,Cajero 102,196
...,...,...
226,Cajero 95,229
227,Cajero 96,202
228,Cajero 97,220
229,Cajero 98,215


In [27]:
tarde = retiro5[(retiro5['Segundos']>=12*3600)].groupby('Cajero')['Día'].count().reset_index()
tarde

,Cajero,Día
0,Cajero 1,87
1,Cajero 10,209
2,Cajero 100,189
3,Cajero 101,219
4,Cajero 102,226
...,...,...
226,Cajero 95,212
227,Cajero 96,213
228,Cajero 97,192
229,Cajero 98,221


In [29]:
comparac = mañana.merge(tarde,on='Cajero')
comparac.rename(columns={'Día_x': 'Cantidad de visitas a cajero en la mañana'}, inplace=True)
comparac.rename(columns={'Día_y': 'Cantidad de visitas a cajero en la tarde'}, inplace=True)
CajerosTarde=comparac[comparac['Cantidad de visitas a cajero en la mañana']<=comparac['Cantidad de visitas a cajero en la tarde']]
CajerosMañana=comparac[comparac['Cantidad de visitas a cajero en la mañana']>comparac['Cantidad de visitas a cajero en la tarde']]

In [30]:
CajerosTarde

,Cajero,Cantidad de visitas a cajero en la mañana,Cantidad de visitas a cajero en la tarde
3,Cajero 101,209,219
4,Cajero 102,196,226
6,Cajero 104,203,203
7,Cajero 105,181,220
11,Cajero 109,202,207
...,...,...,...
223,Cajero 92,205,205
224,Cajero 93,208,219
225,Cajero 94,195,210
227,Cajero 96,202,213


In [31]:
CajerosMañana

,Cajero,Cantidad de visitas a cajero en la mañana,Cantidad de visitas a cajero en la tarde
0,Cajero 1,111,87
1,Cajero 10,214,209
2,Cajero 100,217,189
5,Cajero 103,214,178
8,Cajero 106,217,188
...,...,...,...
219,Cajero 89,228,210
220,Cajero 9,228,202
226,Cajero 95,229,212
228,Cajero 97,220,192
